In [1]:
#loading packages

#install.packages("MASS")
#install.packages("httr")
#install.packages("ggridges")
#install.packages("car")
#install.packages("rsample")
#install.packages("glmnet")
#install.packages("cowplot")
#install.packages("broom")
#install.packages("mltools")
#install.packages("tidyverse")
library(MASS)
library(httr)
#library(ggridges)
library(car)
#library(rsample)
library(glmnet)
#library(cowplot)
#library(broom)
library(mltools)
library(tidyverse)
options(repr.matrix.max.rows = 8)

Warning message:
"package 'MASS' was built under R version 4.4.3"
Warning message:
"package 'httr' was built under R version 4.4.3"
Loading required package: carData

Warning message:
"package 'glmnet' was built under R version 4.4.3"
Loading required package: Matrix

Loaded glmnet 4.1-8

Warning message:
"package 'mltools' was built under R version 4.4.3"
Warning message:
"package 'tidyverse' was built under R version 4.4.3"
── Attaching core tidyverse packages ──────────────────────────────────────────────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ tidyr::expand()     masks Matrix::expand()
✖ dplyr::filter()     masks stats::filter()
✖ dplyr::lag()        masks stats::lag()
✖ tidyr::pack()

In [2]:
#loading in data for london, rome, and budapest listings from the web
url1 <- 'https://www.kaggle.com/api/v1/datasets/download/thedevastator/airbnb-prices-in-european-cities/london_weekdays.csv'
url2 <- 'https://www.kaggle.com/api/v1/datasets/download/thedevastator/airbnb-prices-in-european-cities/london_weekends.csv'
url3 <- 'https://www.kaggle.com/api/v1/datasets/download/thedevastator/airbnb-prices-in-european-cities/rome_weekdays.csv'
url4 <- 'https://www.kaggle.com/api/v1/datasets/download/thedevastator/airbnb-prices-in-european-cities/rome_weekends.csv'
url5 <- 'https://www.kaggle.com/api/v1/datasets/download/thedevastator/airbnb-prices-in-european-cities/budapest_weekdays.csv'
url6 <- 'https://www.kaggle.com/api/v1/datasets/download/thedevastator/airbnb-prices-in-european-cities/budapest_weekends.csv'

download <- function(url) {
    temp <- tempfile(fileext = '.csv')
    GET(url, write_disk(temp))
    data <- read_delim(temp, skip = 0, delim = ",")
    return(data)
}

london_day <- download(url1)
london_end <- download(url2)
rome_day <- download(url3)
rome_end <- download(url4)
budapest_day <- download(url5)
budapest_end <- download(url6)

New names:
• `` -> `...1`
Rows: 4614 Columns: 20
── Column specification ────────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: ","
chr  (1): room_type
dbl (16): ...1, realSum, person_capacity, multi, biz, cleanliness_rating, gu...
lgl  (3): room_shared, room_private, host_is_superhost

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
New names:
• `` -> `...1`
Rows: 5379 Columns: 20
── Column specification ────────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: ","
chr  (1): room_type
dbl (16): ...1, realSum, person_capacity, multi, biz, cleanliness_rating, gu...
lgl  (3): room_shared, room_private, host_is_superhost

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
New names:

In [3]:
#helper function that manually creates the city and weekend variables, also drops any observations with missing values
add_vars <- function(data, city, weekend) {
    data <- mutate(data, 
                   "city" = city,
                   "weekend" = weekend) |>
            drop_na() 
    return(data)
}

#combining into one dataset
london_day_vars <- add_vars(london_day, "London", 0)
london_end_vars <- add_vars(london_end, "London", 1)
rome_day_vars <- add_vars(rome_day, "Rome", 0)
rome_end_vars <- add_vars(rome_end, "Rome", 1)
budapest_day_vars <- add_vars(budapest_day, "Budapest", 0)
budapest_end_vars <- add_vars(budapest_end, "Budapest", 1)
airbnb_raw <- bind_rows(london_day_vars, london_end_vars, rome_day_vars, rome_end_vars, budapest_day_vars, budapest_end_vars)

#removing unnecessary variables, creating temporary variable "ID"
airbnb <- airbnb_raw |> 
    select(-c(1, room_shared, room_private)) |>
    mutate("ID" = row_number())

#splitting data into three sets - one for training, one for validation, one for testing
set.seed(9999)
airbnb_training <- slice_sample(airbnb, prop = 0.6)   # 60% of the data
airbnb_remaining <- anti_join(airbnb, airbnb_training, by = "ID")
airbnb_validation <- slice_sample(airbnb_remaining, prop = 0.5)   # 20% of the data
airbnb_testing <- anti_join(airbnb_remaining, airbnb_validation, by = "ID")   # 20% of the data

#dropping ID
airbnb_training <- select(airbnb_training, -ID)
airbnb_validation <- select(airbnb_validation, -ID)
airbnb_testing <- select(airbnb_testing, -ID)

airbnb_training

realSum,room_type,person_capacity,host_is_superhost,multi,biz,cleanliness_rating,guest_satisfaction_overall,bedrooms,dist,metro_dist,attr_index,attr_index_norm,rest_index,rest_index_norm,lng,lat,city,weekend
<dbl>,<chr>,<dbl>,<lgl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>
92.11418,Private room,2,FALSE,0,0,10,100,1,4.882852,2.0903951,244.9399,5.427916,668.2052,14.54871,12.4575,41.87234,Rome,0
180.00658,Entire home/apt,6,FALSE,0,0,9,92,1,1.845461,0.4056151,326.7423,19.485181,274.6771,20.79940,19.0300,47.50100,Budapest,0
251.56056,Entire home/apt,5,FALSE,0,1,9,95,1,2.239840,0.5201043,645.0867,14.295249,1616.5255,35.19631,12.4770,41.90800,Rome,0
195.91799,Entire home/apt,3,TRUE,0,0,9,99,1,2.261410,0.1300020,575.5465,12.754225,1063.6974,23.15969,12.4870,41.88400,Rome,0
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
195.9180,Entire home/apt,4,FALSE,0,1,10,95,1,2.802646,1.6872206,629.0960,13.94089,2356.1297,51.29958,12.47081,41.89166,Rome,0
251.0930,Entire home/apt,5,FALSE,0,1,9,92,1,2.085412,0.3675287,670.2653,14.85321,1559.5123,33.95497,12.47900,41.90800,Rome,0
167.6897,Private room,2,TRUE,1,0,10,99,1,2.598554,0.3229886,411.9052,28.63199,690.4995,12.36397,-0.10128,51.49167,London,0
177.1866,Entire home/apt,4,FALSE,0,0,9,90,0,1.751475,0.1475408,174.2593,10.40048,300.5346,26.27212,19.06915,47.51064,Budapest,1


In [4]:
#checking multicollinearity
options(repr.matrix.max.rows = 18)
pre_vif <- vif(lm(realSum ~., data = airbnb_training))
pre_vif

airbnb_training <- select(airbnb_training, -c(attr_index_norm, rest_index_norm, lng, lat))
airbnb_validation <- select(airbnb_validation, -c(attr_index_norm, rest_index_norm, lng, lat))
airbnb_testing <- select(airbnb_testing, -c(attr_index_norm, rest_index_norm, lng, lat))

post_vif <- vif(lm(realSum ~., data = airbnb_training))
post_vif

,GVIF,Df,GVIF^(1/(2*Df))
room_type,1.703424e+00,2,1.142433
person_capacity,2.182325e+00,1,1.477269
host_is_superhost,1.149242e+00,1,1.072027
multi,1.439391e+00,1,1.199746
biz,1.598287e+00,1,1.264234
cleanliness_rating,2.323325e+00,1,1.524246
guest_satisfaction_overall,2.407902e+00,1,1.551742
bedrooms,1.572273e+00,1,1.253903
dist,4.797324e+00,1,2.190279
metro_dist,2.177066e+00,1,1.475488


,GVIF,Df,GVIF^(1/(2*Df))
room_type,1.679058,2,1.138325
person_capacity,2.181159,1,1.476875
host_is_superhost,1.145996,1,1.070512
multi,1.437650,1,1.199020
biz,1.593196,1,1.262219
cleanliness_rating,2.319002,1,1.522827
guest_satisfaction_overall,2.405313,1,1.550907
bedrooms,1.571290,1,1.253511
dist,3.701048,1,1.923811
metro_dist,1.928981,1,1.388877


In [5]:
#full and null models
null_model <- lm(realSum ~ 1, data = airbnb_training)
full_model <- lm(realSum ~ ., data = airbnb_training)
training_matrix <- model.matrix(object = realSum ~ ., data = airbnb_training)[, -1]

In [6]:
#stepwise training
n <- nrow(airbnb_training)
stepwise_model <- stepAIC(full_model, direction = "backward", scope = list(lower = null_model, upper = full_model), k = log(n), trace = 0)
summary(stepwise_model)


Call:
lm(formula = realSum ~ room_type + person_capacity + bedrooms + 
    dist + attr_index + city, data = airbnb_training)

Residuals:
    Min      1Q  Median      3Q     Max 
 -620.9   -93.9   -28.3    49.9 12660.7 

Coefficients:
                        Estimate Std. Error t value Pr(>|t|)    
(Intercept)             12.98938   11.95500   1.087 0.277267    
room_typePrivate room -125.48429    6.76781 -18.541  < 2e-16 ***
room_typeShared room  -150.74375   42.89725  -3.514 0.000443 ***
person_capacity         18.58496    2.91564   6.374 1.90e-10 ***
bedrooms               104.43279    5.55834  18.788  < 2e-16 ***
dist                   -17.65810    1.42245 -12.414  < 2e-16 ***
attr_index               0.13856    0.01383  10.017  < 2e-16 ***
cityLondon             306.58317    9.66328  31.727  < 2e-16 ***
cityRome                39.11397    9.15215   4.274 1.94e-05 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 307.8 on 13816 degrees

In [7]:
#LASSO training
lasso_model <- cv.glmnet(x = training_matrix, y = airbnb_training$realSum, alpha = 1)
coef(lasso_model, s = "lambda.min")
lasso_model$lambda.min

17 x 1 sparse Matrix of class "dgCMatrix"
                                      s1
(Intercept)                 -58.75863800
room_typePrivate room      -122.10881709
room_typeShared room       -142.22410629
person_capacity              18.22726884
host_is_superhostTRUE        -9.48745381
multi                         0.28026860
biz                           5.30806750
cleanliness_rating            0.01453976
guest_satisfaction_overall    0.64833233
bedrooms                    104.91726671
dist                        -12.35759477
metro_dist                  -11.17855602
attr_index                    0.10635658
rest_index                    0.02885870
cityLondon                  285.68056561
cityRome                     20.21914342
weekend                      10.69876343

[1] 0.3493718

In [8]:
#ridge training
ridge_model <- cv.glmnet(x = training_matrix, y = airbnb_training$realSum, alpha = 0)
coef(ridge_model, s = "lambda.min")
ridge_model$lambda.min

17 x 1 sparse Matrix of class "dgCMatrix"
                                      s1
(Intercept)                 -59.65382437
room_typePrivate room      -116.19795603
room_typeShared room       -142.88522412
person_capacity              18.96307291
host_is_superhostTRUE       -11.72463250
multi                         0.85492757
biz                           6.29450250
cleanliness_rating            0.53892311
guest_satisfaction_overall    0.59946916
bedrooms                    102.98316847
dist                         -9.68911871
metro_dist                  -13.78570684
attr_index                    0.10864913
rest_index                    0.03504473
cityLondon                  263.11056063
cityRome                      4.31505310
weekend                      11.49273389

[1] 8.455245

In [9]:
#validation
validation_matrix <- model.matrix(object = realSum ~ ., data = airbnb_validation)[, -1]
stepwise_preds <- predict(stepwise_model, newdata = airbnb_validation)
lasso_preds <- predict(lasso_model, newx = validation_matrix, s = "lambda.min")
ridge_preds <- predict(ridge_model, newx = validation_matrix, s = "lambda.min")

validation_rmse <- tibble(
    Model = c("Multiple Linear Regression with Stepwise Selection", "LASSO Regression with Minimum MSE", "Ridge Regression with Minimum MSE"),
    RMSE = c(rmse(stepwise_preds, airbnb_validation$realSum), rmse(lasso_preds, airbnb_validation$realSum), rmse(ridge_preds, airbnb_validation$realSum))
)

validation_rmse

Model,RMSE
<chr>,<dbl>
Multiple Linear Regression with Stepwise Selection,243.5199
LASSO Regression with Minimum MSE,243.3032
Ridge Regression with Minimum MSE,243.2194


In [10]:
#for comparison
sd(airbnb_training$realSum)

[1] 345.8239

In [11]:
#testing
testing_matrix <- model.matrix(object = realSum ~ ., data = airbnb_testing)[, -1]

#uncomment RMSE for the chosen model
#final_preds <- predict(stepwise_model, newdata = airbnb_testing)
#final_preds <- predict(lasso_model, newx = testing_matrix, s = "lambda.min")
#final_preds <- predict(ridge_model, newx = testing_matrix, s = "lambda.min")
#rmse(final_preds, airbnb_testing$realSum)